In [1]:
import json
from langchain_community.vectorstores import FAISS
# from langchain_community.embeddings import OllamaEmbeddings

from langchain_ollama import OllamaEmbeddings

from langchain_community.llms import Ollama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

### Step 1: Load and Process the Documents

In [ ]:
print("Step 1: Loading and Processing Documents...")
try:
    with open('project_1_publications.json', 'r') as f:
        data = json.load(f)
except FileNotFoundError:
    print("Error: 'project_1_publications.json' not found.")
    print("Please make sure the JSON file is in the same directory as this script.")
    exit()



Step 1: Loading and Processing Documents...


In [ ]:
# Extract the publication descriptions and create LangChain Document objects
# This follows the principle of reading our files first.
documents = []

for item in data:
    content = item.get("publication_description", "")
    metadata = {"title": item.get("title", "No Title")}
    documents.append(Document(page_content=content, metadata=metadata))

In [4]:
# Divide the texts into chunks so they can be fed into your embedding model.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

print(f"Loaded and split {len(docs)} document chunks.")


Loaded and split 1207 document chunks.


### Step 2: Create Embeddings and Store in Vector Database (FAISS) 

In [ ]:


print("\nStep 2: Creating Embeddings and Storing in FAISS...")
# Initialize Ollama embeddings using the Mistral model
# This step embeds the chunked texts into vectors.
ollama_embeddings = OllamaEmbeddings(model="mistral")

# Create a FAISS vector store from the document chunks and their embeddings.
# This pushes the vectors and text to the database.
# FAISS is a local vector store, so it saves the index to your disk.
vector = FAISS.from_documents(docs, ollama_embeddings)
print("Embeddings created and stored in FAISS.")


Step 2: Creating Embeddings and Storing in FAISS...
Embeddings created and stored in FAISS.


### Step 3: Initialize the LLM and Create the RAG Chain |

In [ ]:

print("\nStep 3: Initializing LLM and Creating RAG Chain...")
# Initialize the Ollama LLM with the Mistral model
llm = Ollama(model="mistral")

# Create a prompt template. This structures the input for the LLM.
# The goal is to pass the answers from the vector database to your LLM.
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
Think step by step.
If you don't know the answer, just say that you don't know.
<context>
{context}
</context>
Question: {input}""")

# Create the main chain that combines document retrieval and question answering
document_chain = create_stuff_documents_chain(llm, prompt)

# Create the retrieval chain. This component will retrieve documents from the
# vector store before passing them to the document_chain.
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)
print("RAG chain created successfully.")


Step 3: Initializing LLM and Creating RAG Chain...
RAG chain created successfully.


C:\Users\gouri\AppData\Local\Temp\ipykernel_20128\3396880923.py:5: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="mistral")


# Step 4: Ask Questions and Get Answers

In [ ]:


print("\nStep 4: Ready to Answer Questions!")
print("Type 'exit' to quit.")

while True:
    try:
        user_input = input("\nYour Question: ")
        if user_input.lower() == 'exit':
            print("Exiting assistant. Goodbye!")
            break

        # Invoke the chain with the user's question
        response = retrieval_chain.invoke({"input": user_input})

        # Print the answer
        print("\nAssistant's Answer:")
        print(response["answer"])

    except Exception as e:
        print(f"An error occurred: {e}")


Step 4: Ready to Answer Questions!
Type 'exit' to quit.

Assistant's Answer:
 In the provided context, there is no information about defining a custom agent for a specific task such as a game or a chatbot. However, if we are talking about a Neural Network model (like NBeats), here's how you might define a custom agent:

1. Identify the problem you want to solve with your custom agent. This could be anything from playing a game like chess or Go, to controlling a robot, to generating text in a chatbot.

2. Choose an appropriate architecture for your model based on the problem you're trying to solve. For instance, if it's a sequence-to-sequence task like text generation, you might choose an LSTM or Transformer architecture. If it's a game with a large state space, you might consider using a deep reinforcement learning model.

3. Prepare your dataset and labels for training the agent. This might involve collecting data from the environment, cleaning the data, and possibly preprocessing it

In [9]:
user_input_2 = "What’s the role of memory in LangGraph?"
response_2 = retrieval_chain.invoke({"input": user_input_2})

print("\nAssistant's Answer:")
print(response_2["answer"])


Assistant's Answer:
 The provided context does not explicitly explain the role of memory in LangGraph as it only discusses ChatGPT's use of memory. However, considering that Langchain-OpenAI and knowledge graphs share similarities with how ChatGPT functions (by using connections between entities), we can infer that memory in LangGraph would play a role in enabling the platform to remember relationships between various entities (films, actors, directors, etc.) and use this information to answer complex queries efficiently.

For a more accurate answer, it would be best to consult documentation or research about Langchain-OpenAI specifically.
